In [1]:
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
OSM = pd.read_json('amenities-vancouver.json.gz', lines=True)
OSM

,lat,lon,timestamp,amenity,name,tags
0,49.260812,-123.125736,2020-03-20T18:22:12.000-07:00,cafe,Starbucks,"{'brand:wikidata': 'Q37158', 'official_name': ..."
1,49.260953,-123.125704,2019-08-02T18:11:20.000-07:00,fast_food,Salad Loop,{'opening_hours': 'Mo-Fr 07:00-17:00; Sa 10:00...
2,49.373423,-123.291894,2016-10-10T02:14:29.000-07:00,toilets,NaN,{}
3,49.249848,-122.959708,2011-09-06T03:52:10.000-07:00,bbq,NaN,{}
4,49.370898,-123.280448,2015-05-03T00:42:25.000-07:00,place_of_worship,St. Monica's Anglican Church,"{'addr:housenumber': '6404', 'addr:street': 'W..."
...,...,...,...,...,...,...
17713,49.278424,-122.806704,2013-03-26T23:45:49.000-07:00,cafe,Creekside Coffee,{}
17714,49.278770,-122.797628,2013-03-26T23:45:49.000-07:00,restaurant,Togo Sushi,{'cuisine': 'japanese'}
17715,49.276443,-122.790138,2013-03-26T23:45:50.000-07:00,parking,NaN,{}
17716,49.282666,-122.826978,2019-09-13T13:56:49.000-07:00,pub,Brown's Social House,"{'addr:housenumber': '215', 'brewery': 'Guinne..."


In [2]:
OSM.amenity.unique()

array(['cafe', 'fast_food', 'toilets', 'bbq', 'place_of_worship',
       'post_box', 'telephone', 'fuel', 'vending_machine', 'restaurant',
       'parking_entrance', 'pub', 'bicycle_parking', 'school', 'bench',
       'community_centre', 'waste_basket', 'pharmacy', 'dentist',
       'doctors', 'post_office', 'atm', 'childcare', 'parking',
       'public_building', 'bank', 'cinema', 'theatre', 'ferry_terminal',
       'bar', 'library', 'car_rental', 'drinking_water', 'shelter',
       'car_sharing', 'bicycle_rental', 'clinic', 'recycling',
       'public_bookcase', 'university', 'dojo', 'food_court',
       'seaplane terminal', 'arts_centre', 'ice_cream',
       'charging_station', 'car_wash', 'fountain', 'veterinary',
       'bicycle_repair_station', 'waste_disposal', 'photo_booth',
       'luggage_locker', 'bureau_de_change', 'parking_space', 'nightclub',
       'social_facility', 'taxi', 'bus_station', 'college',
       'construction', 'post_depot', 'nursery', 'clock', 'kindergarten'

In [3]:
rest_list = ['cafe', 'fast_food', 'bbq', 'restaurant', 'pub',
       'bar', 'ice_cream', 'bistro', 'juice_bar']
restaurants = OSM[OSM.amenity.isin(rest_list)]

In [8]:
OSM[OSM.name.isin(['Orange Julius'])]

,lat,lon,timestamp,amenity,name,tags
1673,49.227280,-123.000180,2018-04-21T18:56:31.000-07:00,bar,Orange Julius,"{'level': '0', 'name:en': 'Orange Julius'}"
1749,49.225784,-122.997807,2019-08-24T15:19:56.000-07:00,fast_food,Orange Julius,"{'brand:wikidata': 'Q3355059', 'level': '1', '..."
1838,49.250704,-122.896108,2017-03-13T04:15:38.000-07:00,bar,Orange Julius,{'name:en': 'Orange Julius'}
2682,49.186848,-122.849032,2019-06-25T03:54:28.000-07:00,fast_food,Orange Julius,"{'brand:wikidata': 'Q3355059', 'brand:wikipedi..."
6028,49.324905,-123.136894,2019-04-29T03:53:05.000-07:00,juice_bar,Orange Julius,"{'wheelchair': 'yes', 'shop': 'beverages', 'le..."
10084,49.279597,-123.117415,2019-08-29T17:37:11.000-07:00,fast_food,Orange Julius,"{'brand:wikidata': 'Q3355059', 'addr:housenumb..."
11402,49.114470,-122.675202,2014-08-07T06:09:36.000-07:00,fast_food,Orange Julius,{}
11613,49.197116,-123.176786,2019-09-14T05:00:22.000-07:00,fast_food,Orange Julius,"{'brand:wikidata': 'Q3355059', 'cuisine': 'jui..."


In [5]:
#pd.set_option('display.max_rows', 100)
restaurants[restaurants.name.isnull()]

,lat,lon,timestamp,amenity,name,tags
3,49.249848,-122.959708,2011-09-06T03:52:10.000-07:00,bbq,NaN,{}
786,49.049771,-122.319001,2019-09-02T22:08:26.000-07:00,fast_food,NaN,"{'official_name': 'Kami Sushi Enterprises', 'a..."
1263,49.237670,-122.782747,2012-03-01T05:24:27.000-08:00,bbq,NaN,{}
1553,49.263266,-123.110529,2014-07-11T19:59:54.000-07:00,pub,NaN,{}
2046,49.229367,-123.004155,2017-03-22T04:32:05.000-07:00,bar,NaN,{'addr:housenumber': '1822'}
...,...,...,...,...,...,...
16520,49.050337,-122.800687,2014-05-17T04:16:00.000-07:00,restaurant,NaN,{'cuisine': 'sushi'}
16667,49.200029,-122.911494,2013-03-03T22:23:02.000-08:00,fast_food,NaN,"{'level': '1', 'cuisine': 'seafood'}"
17089,49.312247,-122.925978,2019-07-15T07:01:57.000-07:00,cafe,NaN,{}
17628,49.139018,-122.889496,2020-03-21T05:16:15.000-07:00,restaurant,NaN,{'cuisine': 'thai'}


In [6]:
restaurants[~restaurants.name.isnull()]

,lat,lon,timestamp,amenity,name,tags
0,49.260812,-123.125736,2020-03-20T18:22:12.000-07:00,cafe,Starbucks,"{'brand:wikidata': 'Q37158', 'official_name': ..."
1,49.260953,-123.125704,2019-08-02T18:11:20.000-07:00,fast_food,Salad Loop,{'opening_hours': 'Mo-Fr 07:00-17:00; Sa 10:00...
13,49.126650,-123.182470,2020-03-30T09:08:51.000-07:00,restaurant,Best Bite Indian Cuisine,"{'addr:housenumber': '10-3891', 'phone': '+1-6..."
16,49.283192,-123.109050,2015-12-18T21:41:07.000-08:00,pub,The Cambie,"{'toilets:wheelchair': 'no', 'wheelchair': 'li..."
19,49.265951,-123.246630,2011-11-19T08:06:36.000-08:00,pub,Mahony and Sons,{'wheelchair': 'yes'}
...,...,...,...,...,...,...
17712,49.250408,-123.076261,2017-07-08T05:22:57.000-07:00,restaurant,House of Dosas,"{'addr:housenumber': '1391', 'phone': '+1-604-..."
17713,49.278424,-122.806704,2013-03-26T23:45:49.000-07:00,cafe,Creekside Coffee,{}
17714,49.278770,-122.797628,2013-03-26T23:45:49.000-07:00,restaurant,Togo Sushi,{'cuisine': 'japanese'}
17716,49.282666,-122.826978,2019-09-13T13:56:49.000-07:00,pub,Brown's Social House,"{'addr:housenumber': '215', 'brewery': 'Guinne..."


In [7]:
print(len(restaurants.name.unique()))
restaurants.name.unique()

3475


array(['Starbucks', 'Salad Loop', nan, ..., 'House of Dosas',
       'Creekside Coffee', "Gallagher's Cafe"], dtype=object)